In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype as is_datetime

REQUIRED_COLS = {'거래일자', '품목명', '반입량', '금액', '전년 반입량', '전년 금액'}

def to_numeric_clean(s: pd.Series) -> pd.Series:
    if pd.api.types.is_numeric_dtype(s):
        return s
    return pd.to_numeric(
        s.astype(str)
         .str.replace(",", "", regex=False)
         .str.replace(" ", "", regex=False)
         .str.replace("%", "", regex=False)
         .str.replace("\u2212", "-", regex=False),
        errors="coerce"
    )

def preprocess_for_fill(df: pd.DataFrame, date_col: str|None=None, item_col: str|None=None,
                        add_calendar_cols: bool=True, coerce_numeric: bool=True) -> pd.DataFrame:
    out = df.copy()

    if date_col is None:
        cands = [c for c in out.columns if ("거래일자" in str(c)) or ("date" in str(c).lower())]
        if not cands:
            raise ValueError("`거래일자`(또는 date 유사) 컬럼을 찾을 수 없습니다. date_col을 명시하세요.")
        date_col = cands[0]
    if date_col != '거래일자' and date_col in out.columns:
        out = out.rename(columns={date_col: '거래일자'})
    elif '거래일자' not in out.columns:
        raise ValueError(f"날짜 컬럼 '{date_col}'(또는 '거래일자')을 찾을 수 없습니다.")

    if item_col is None:
        icands = [c for c in out.columns if ("품목" in str(c)) or ("품종" in str(c)) or ("품목명" in str(c))]
        item_col = icands[0] if icands else '품목명'
    if item_col != '품목명' and item_col in out.columns:
        out = out.rename(columns={item_col: '품목명'})
    if '품목명' not in out.columns:
        out['품목명'] = "미상"

    if not is_datetime(out['거래일자']):
        out['거래일자'] = pd.to_datetime(out['거래일자'], errors='coerce')

    if coerce_numeric:
        for c in ['반입량','금액','전년 반입량','전년 금액']:
            if c in out.columns:
                out[c] = to_numeric_clean(out[c])

    if add_calendar_cols and '거래일자' in out.columns:
        out['연도'] = out['거래일자'].dt.year
        out['월'] = out['거래일자'].dt.month
        out['weekday'] = out['거래일자'].dt.weekday
        out['is_sunday'] = (out['weekday'] == 6)
        out['주차'] = out['거래일자'].dt.isocalendar().week.astype('Int64')

    return out

def _aggregate_keys_for_next(src: pd.DataFrame, agg: str) -> pd.DataFrame:
    if agg == 'mean':
        return (src.groupby(['거래일자','품목명'], as_index=False)
                  .agg({'다음해_전년반입량':'mean','다음해_전년금액':'mean'}))
    sorter = src.sort_values(['거래일자','품목명'])
    if agg == 'first':
        return sorter.groupby(['거래일자','품목명'], as_index=False).first()
    return sorter.groupby(['거래일자','품목명'], as_index=False).last()

def read_csv_smart(path: str) -> pd.DataFrame:
    for enc in ['utf-8-sig','cp949','euc-kr']:
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            pass
    raise RuntimeError(f"CSV 로드 실패: {path}")

def fill_from_nextyear_seq(df: pd.DataFrame, year_start: int=2024, year_end: int=2015,
                           agg: str='last', fix_negative_amount: bool=True, verbose: bool=True,
                           date_col: str|None=None, item_col: str|None=None,
                           add_calendar_cols: bool=True, coerce_numeric: bool=True):
    out = preprocess_for_fill(df, date_col, item_col, add_calendar_cols, coerce_numeric)

    missing = REQUIRED_COLS - set(out.columns)
    if missing:
        raise ValueError(f"다음 컬럼이 없습니다: {sorted(missing)}")

    if fix_negative_amount:
        out.loc[out['금액'] < 0, '금액'] = pd.NA

    stats_rows, log_rows = [], []
    for y in range(year_start, year_end - 1, -1):
        ymask = out['거래일자'].dt.year == y
        if not ymask.any():
            if verbose: print(f"[{y}] 대상 행 없음")
            continue

        src = out[['거래일자','품목명','전년 반입량','전년 금액']].copy()
        src['거래일자'] = src['거래일자'] - pd.DateOffset(years=1)
        src = src.rename(columns={'전년 반입량':'다음해_전년반입량','전년 금액':'다음해_전년금액'})
        src = src[src['거래일자'].dt.year == y]
        src = _aggregate_keys_for_next(src, agg)

        cur = out[ymask].merge(src, on=['거래일자','품목명'], how='left', validate='m:1')
        cur['반입량_before'] = cur['반입량']
        cur['금액_before'] = cur['금액']

        b_qty = cur['반입량'].isna().sum()
        b_amt = cur['금액'].isna().sum()

        m_qty = cur['반입량'].isna() & cur['다음해_전년반입량'].notna()
        m_amt = cur['금액'].isna()   & cur['다음해_전년금액'].notna()

        if m_qty.any():
            tmp = cur.loc[m_qty, ['거래일자','품목명','반입량_before','다음해_전년반입량']].copy()
            tmp['연도'] = y; tmp['필드'] = '반입량'
            tmp['채우기_이후값'] = tmp['다음해_전년반입량']
            tmp['공여자_다음해_원래거래일자'] = tmp['거래일자'] + pd.DateOffset(years=1)
            tmp = tmp.rename(columns={'반입량_before':'채우기_이전값','다음해_전년반입량':'공여자_전년도값'})
            log_rows.append(tmp)

        if m_amt.any():
            tmp = cur.loc[m_amt, ['거래일자','품목명','금액_before','다음해_전년금액']].copy()
            tmp['연도'] = y; tmp['필드'] = '금액'
            tmp['채우기_이후값'] = tmp['다음해_전년금액']
            tmp['공여자_다음해_원래거래일자'] = tmp['거래일자'] + pd.DateOffset(years=1)
            tmp = tmp.rename(columns={'금액_before':'채우기_이전값','다음해_전년금액':'공여자_전년도값'})
            log_rows.append(tmp)

        cur.loc[m_qty, '반입량'] = cur.loc[m_qty, '다음해_전년반입량']
        cur.loc[m_amt, '금액']   = cur.loc[m_amt, '다음해_전년금액']

        a_qty = cur['반입량'].isna().sum()
        a_amt = cur['금액'].isna().sum()
        out.loc[ymask, ['반입량','금액']] = cur[['반입량','금액']].values

        filled_cnt = int((m_qty | m_amt).sum())
        if verbose:
            print(f"[{y}] 반입량 NaN {b_qty}→{a_qty} (채움 {b_qty - a_qty}),  "
                  f"금액 NaN {b_amt}→{a_amt} (채움 {b_amt - a_amt}),  보강 행수 {filled_cnt}")

        stats_rows.append({'year': y, 'filled_rows': filled_cnt,
                           'qty_before': int(b_qty), 'qty_after': int(a_qty),
                           'amt_before': int(b_amt), 'amt_after': int(a_amt)})

    stats_df = pd.DataFrame(stats_rows, columns=['year','filled_rows','qty_before','qty_after','amt_before','amt_after'])

    if log_rows:
        fill_log_df = (pd.concat(log_rows, ignore_index=True)
                         [['연도','거래일자','품목명','필드','채우기_이전값','채우기_이후값','공여자_전년도값','공여자_다음해_원래거래일자']])
    else:
        fill_log_df = pd.DataFrame(columns=['연도','거래일자','품목명','필드','채우기_이전값','채우기_이후값','공여자_전년도값','공여자_다음해_원래거래일자'])

    return out, stats_df, fill_log_df

def weekly_avg_unit_price(
    df: pd.DataFrame,
    by_item: bool = True,                 # 품목별 집계 여부
    use_existing_week_cols: bool = True,  # df의 '연도','주차'를 사용할지
    drop_negative_amount: bool = True,    # 음수 금액 제외
    return_weighted: bool = False         # 물량가중 평균단가(주차 총금액/총반입량) 포함 여부
) -> pd.DataFrame:
    x = df.copy()

    # 날짜/결측 처리
    if '거래일자' not in x.columns:
        raise ValueError("'거래일자' 컬럼 필요")
    if not is_datetime(x['거래일자']):
        x['거래일자'] = pd.to_datetime(x['거래일자'], errors='coerce')
    if drop_negative_amount and '금액' in x.columns:
        x = x[~(x['금액'] < 0)]

    # 주차 키
    if use_existing_week_cols and {'연도','주차'} <= set(x.columns):
        x['_year'] = x['연도']
        x['_week'] = x['주차'].astype('Int64')
    else:
        iso = x['거래일자'].dt.isocalendar()
        x['_year'] = iso.year.astype(int)
        x['_week'] = iso.week.astype(int)

    # (품목, 일자) 합산 → 일별 금액/반입량
    group_day = ['거래일자']
    if by_item:
        group_day.insert(0, '품목명')
    day = (x.groupby(group_day, as_index=False)
             .agg(금액=('금액','sum'), 반입량=('반입량','sum')))

    # 안전한 일별 단가 (반입량>0 & 금액 존재)
    day = day[(day['반입량'].notna()) & (day['반입량'] > 0) & (day['금액'].notna())]
    day['일별단가'] = day['금액'] / day['반입량']

    # 일자 → 주차 키 부여
    if use_existing_week_cols and {'연도','주차'} <= set(x.columns):
        key_cols = ['거래일자','_year','_week'] if not by_item else ['거래일자','품목명','_year','_week']
        day = day.merge(x[key_cols].drop_duplicates(),
                        on=(['거래일자','품목명'] if by_item else ['거래일자']),
                        how='left')
    else:
        iso_day = day['거래일자'].dt.isocalendar()
        day['_year'] = iso_day.year.astype(int)
        day['_week'] = iso_day.week.astype(int)

    # 주차 집계
    group_week = ['_year','_week'] if not by_item else ['품목명','_year','_week']
    weekly = (day.groupby(group_week, as_index=False)
                .agg(
                    주차_일수=('거래일자','nunique'),
                    주간_평균단가=('일별단가','mean'),   # ← 이것만 남김
                    금액_합=('금액','sum'),
                    반입량_합=('반입량','sum')
                ))

    # (옵션) 물량가중 평균단가
    if return_weighted:
        weekly['주간_물량가중_단가'] = weekly['금액_합'] / weekly['반입량_합']

    # 마무리
    weekly = weekly.rename(columns={'_year':'연도','_week':'주차'})
    order_cols = ['연도','주차'] if not by_item else ['품목명','연도','주차']

    out_cols = order_cols + ['주차_일수','주간_평균단가']
    if return_weighted:
        out_cols += ['주간_물량가중_단가']
    # 참고용 합계가 필요 없으면 다음 두 줄 주석 해제로 제거 가능
    # weekly = weekly.drop(columns=['금액_합','반입량_합'])
    out_cols += ['금액_합','반입량_합']  # 합계도 보고 싶으면 유지

    return weekly[out_cols].sort_values(order_cols).reset_index(drop=True)

def run_fill_pipeline(path_in: str, path_out: str, enc_out: str="utf-8-sig",
                      year_start: int=2024, year_end: int=2015, agg: str="last",
                      fix_negative_amount: bool=True, save_weekly: bool=True, verbose: bool=True):
    pin, pout = Path(path_in), Path(path_out)
    pout.parent.mkdir(parents=True, exist_ok=True)

    df_in = read_csv_smart(str(pin))

    df_filled, yearly_stats, fill_log = fill_from_nextyear_seq(
        df_in, year_start=year_start, year_end=year_end,
        agg=agg, fix_negative_amount=fix_negative_amount, verbose=verbose
    )

    df_filled.to_csv(pout, index=False, encoding=enc_out)
    yearly_stats.to_csv(pout.with_name(pout.stem + "_stats.csv"), index=False, encoding=enc_out)
    fill_log.to_csv(pout.with_name(pout.stem + "_filllog.csv"), index=False, encoding=enc_out)

    weekly_item = weekly_all = None
    if save_weekly:
        weekly_item = weekly_avg_unit_price(df_filled, by_item=True,  use_existing_week_cols=True)
        weekly_all  = weekly_avg_unit_price(df_filled, by_item=False, use_existing_week_cols=True)
        weekly_item.to_csv(pout.with_name(pout.stem + "_weekly_item.csv"), index=False, encoding=enc_out)
        weekly_all.to_csv(pout.with_name(pout.stem + "_weekly_all.csv"), index=False, encoding=enc_out)

    print("[저장 완료]")
    print(" -", pout)
    print(" -", pout.with_name(pout.stem + "_stats.csv"))
    print(" -", pout.with_name(pout.stem + "_filllog.csv"))
    if save_weekly:
        print(" -", pout.with_name(pout.stem + "_weekly_item.csv"))
        print(" -", pout.with_name(pout.stem + "_weekly_all.csv"))

    return df_filled, yearly_stats, fill_log, weekly_item, weekly_all

In [2]:
from pathlib import Path
import traceback

# 1) 입력/출력 폴더
INPUT_DIR = Path(".").resolve()        # 현재 폴더 그대로 사용
OUTPUT_DIR = INPUT_DIR.parent / (INPUT_DIR.name + "_전처리")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"[경로] 입력 폴더 : {INPUT_DIR}")
print(f"[경로] 출력 폴더 : {OUTPUT_DIR}")

# 2) CSV 수집
csv_files = sorted(INPUT_DIR.glob("*.csv"))
if not csv_files:
    raise SystemExit(f"[에러] CSV를 찾지 못했습니다: {INPUT_DIR}")

print(f"[확인] 대상 파일 수: {len(csv_files)}")
for p in csv_files:
    print(" -", p.name)

# 3) 파일별 처리
ok, fail = 0, 0
for file in csv_files:
    try:
        item_name = file.stem
        out_path = OUTPUT_DIR / f"{item_name}_filled.csv"

        print(f"\n▶ {file.name} → {out_path.name}")
        df_filled, yearly_stats, fill_log, weekly_item, weekly_all = run_fill_pipeline(
            str(file),          # 입력 경로
            str(out_path),      # 출력 경로
            "utf-8-sig",        # 인코딩
            year_start=2024,
            year_end=2015,
            agg="last",
            fix_negative_amount=True,
            save_weekly=True,
            verbose=True
        )
        ok += 1
    except Exception as e:
        fail += 1
        print(f"[에러] {file.name} 실패: {e}")
        traceback.print_exc(limit=1)

print(f"\n✅ 완료: 성공 {ok}개, 실패 {fail}개")
print(f"   결과 폴더: {OUTPUT_DIR}")

[경로] 입력 폴더 : /Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/56 수민/cleaned
[경로] 출력 폴더 : /Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/56 수민/cleaned_전처리
[확인] 대상 파일 수: 5
 - 감자_cleaned.csv
 - 고구마_cleaned.csv
 - 단호박_cleaned.csv
 - 적양배추_cleaned.csv
 - 토마토전체_cleaned.csv

▶ 감자_cleaned.csv → 감자_cleaned_filled.csv
[2024] 반입량 NaN 67→67 (채움 0),  금액 NaN 67→67 (채움 0),  보강 행수 0
[2023] 반입량 NaN 62→62 (채움 0),  금액 NaN 62→62 (채움 0),  보강 행수 0
[2022] 반입량 NaN 60→60 (채움 0),  금액 NaN 60→60 (채움 0),  보강 행수 0
[2021] 반입량 NaN 62→62 (채움 0),  금액 NaN 62→61 (채움 1),  보강 행수 1
[2020] 반입량 NaN 61→61 (채움 0),  금액 NaN 61→61 (채움 0),  보강 행수 0
[2019] 반입량 NaN 60→60 (채움 0),  금액 NaN 60→60 (채움 0),  보강 행수 0
[2018] 반입량 NaN 60→60 (채움 0),  금액 NaN 60→60 (채움 0),  보강 행수 0
[2017] 반입량 NaN 60→60 (채움 0),  금액 NaN 60→60 (채움 0),  보강 행수 0
[2016] 반입량 NaN 60→59 (채움 1),  금액 NaN 60→59 (채움 1),  보강 행수 1
[2015] 반입량 NaN 28→28 (채움 0),  금액 NaN 28→28 (채움 0),  보강 행수 0
[저장 완료]
 - /Users/sojinjung/Documents/GitHub/G